### Undersöker relationen mellan gymasielinje och betyg i engelska

In [9]:
import pandas as pd
import plotly.express as px

data_path = "..\project\data\Gymnasieskola - Kursprovsbetyg, elever som började 2011 eller senare VT23 Skolenhet.xlsx"
sheets = [
    "Engelska 5",
    "Engelska 6",
    "Matematik 1A",
    "Matematik 1B",
    "Matematik 2B",
    "Matematik 3B",
    "Matematik 3C",
    "Matematik 4",
    "Svenska 1",
    "Svenska 3"
]
#df_test = pd.read_excel(data_path, sheet_name="Engelska 6", skiprows=8)

df_combined = pd.concat([pd.read_excel(data_path, sheet_name=sheet, skiprows=8) for sheet in sheets], ignore_index=True)

df_combined





,Skola,Skol-enhetskod,Skolkommun,Kommun-kod,Typ av huvudman,Huvudman,Huvudman orgnr,Program,Antal elever,Gnm. provbetp.,Andel F,Andel E,Andel D,Andel C,Andel B,Andel A
0,Lärlingsgymnasiet i Ale,93221031,Ale,1440,Enskild,Lärlingsgymnasiet i Sverige AB,5567628176,Barn- och fritidsprogrammet,5,14,.,..,..,..,..,.
1,Lärlingsgymnasiet i Ale,93221031,Ale,1440,Enskild,Lärlingsgymnasiet i Sverige AB,5567628176,Bygg- och anläggningsprogrammet,17,12.9,.,..,29.4,35.3,..,.
2,Lärlingsgymnasiet i Ale,93221031,Ale,1440,Enskild,Lärlingsgymnasiet i Sverige AB,5567628176,El- och energiprogrammet,6,14.6,.,..,..,..,.,..
3,Lärlingsgymnasiet i Ale,93221031,Ale,1440,Enskild,Lärlingsgymnasiet i Sverige AB,5567628176,Fordons- och transportprogrammet,22,13.3,.,31.8,..,36.4,..,.
4,Lärlingsgymnasiet i Ale,93221031,Ale,1440,Enskild,Lärlingsgymnasiet i Sverige AB,5567628176,Försäljnings- och serviceprogrammet,12,14.8,.,..,..,..,..,..
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9923,Vattenfallgymnasiet,34739511,Östhammar,382,Enskild,Forsmarks Gymnasium AB,5566597885,Teknikprogrammet,11,12.5,.,..,..,..,..,.
9924,Gränsälvsgymnasiet,52690153,Övertorneå,2518,Kommunal,Övertorneå kommun,2120002700,Ekonomiprogrammet,4,..,..,..,..,..,..,..
9925,Gränsälvsgymnasiet,52690153,Övertorneå,2518,Kommunal,Övertorneå kommun,2120002700,Naturvetenskapsprogrammet,3,..,..,..,..,..,..,..
9926,Gränsälvsgymnasiet,52690153,Övertorneå,2518,Kommunal,Övertorneå kommun,2120002700,Samhällsvetenskapsprogrammet,2,..,..,..,..,..,..,..


In [10]:
def bar_plot(xStr:str, yStr:str, titleStr:str, df):
    fig = px.bar(x = df[xStr].str.replace(' ', '<br>'),y = df[yStr], title=titleStr)
    return fig

In [11]:
df_combined['Gnm. provbetp.'] = pd.to_numeric(df_combined['Gnm. provbetp.'], errors='coerce')

average_df = df_combined.groupby('Program')['Gnm. provbetp.'].mean().reset_index()

fig = bar_plot("Program", "Gnm. provbetp.", "Gentomsnittligt nationella prov poäng per gymnasielinje", average_df)
fig.update_layout(
    height = 600,
    width = 1400
)
fig.update_traces(width = 0.5)
fig.update_layout(width=1800, 
                  height=600, 
                  xaxis_tickangle=45,
                  yaxis_title="Poäng",
                  xaxis_title="Gymnasielinje"
                  )


fig.show()

### Huvudmans invärkan på elevers betyg

In [12]:

huvudman_df = df_combined.groupby('Typ av huvudman')['Gnm. provbetp.'].mean().reset_index()
fig = bar_plot("Typ av huvudman", "Gnm. provbetp.", "Genomsnittligt nationella prov per huvudmans typ", huvudman_df)
fig.update_traces(width = 0.7)
fig.update_layout( 
                  yaxis_title="Poäng",
                  xaxis_title="Huvudman",
                  width=800, 
                  height=600,
                  xaxis = dict(linecolor='gray', linewidth=2, showline=True),
                  yaxis = dict(linecolor='gray', linewidth=2, showline=True)
                  )
fig.show()

### Elevers betyg i Engelska 5 över (2012-2017)

In [13]:

base_path = "..\\project\\data\\HT12-HT22"
years = range(12, 23)
file_paths = {
    f"df_HT{year}": f"{base_path}\\Gymnasieskola - Kursprovsbetyg, elever som började 2011 eller senare HT{year} Skolenhet.xlsx"
    for year in years
}



In [14]:
base_path = "..\project\data\HT12-HT22"
file_pattern = "Gymnasieskola - Kursprovsbetyg, elever som började 2011 eller senare HT{year} Skolenhet.xlsx"

# Years range
years = range(12, 18)  # From HT12 to HT17 because after HT17 they change how they format the information

# Dictionary to hold DataFrames
dfs = {}

# Loop through each year, construct file path, and load the DataFrame


sheet_to_extract = "Engelska 5"

for year in years:
    # Construct file path
    file_path = f"{base_path}\{file_pattern.format(year=str(year).zfill(2))}"
    
    # Read the Excel file and store the DataFrame in the dictionary
    dfs[f'HT{year}'] = pd.read_excel(file_path, skiprows=7, sheet_name=sheet_to_extract)



In [17]:
years = range(12, 18)  # From HT12 to HT17


averages = []

for year in years:
    # Getting rid of '..'
    numeric_scores = pd.to_numeric(dfs[f'HT{year}']['Gnm. provbetp.'], errors='coerce')
    
    # Calculating the mean, ignoring NaN values
    avg_score = numeric_scores.mean()
    
    # Append the year and average score to the 'averages' list
    averages.append({'Year': f'HT{year}', 'Average Score': avg_score})


# Convert the 'averages' list to a DataFrame for plotting
average_df = pd.DataFrame(averages)

# Plotting the line chart with Plotly Express
fig = px.line(average_df, x='Year', y='Average Score', title='Engelska betyg över åren', markers=True)


fig.update_layout(
                  xaxis = dict(linecolor='gray', linewidth=2, showline=True),
                  yaxis = dict(linecolor='gray', linewidth=2, showline=True),
                  width=800, 
                  height=600,
                  yaxis_title = "Poäng",
                  xaxis_title="Termin/År",
                  yaxis_range=[10, 15]
                  )
# Show the figure
fig.show()
